## Creați o buclă care cere un PIN de 4 cifre până e corect

explicatie exercitiu: m-am jucat cu chatgpt si am vrut sa protejez parola cumva. la inceput, am avut o eroare legata de bcrypt pt ca nu l-am putut importa. #de aceea, am mers in terminal via view>Ctrl+
#acolo am put 'python --version
#dupa am pus 'pip install bcrypt'. dupa noul cod a functionat. mai jos gasiti explicatiile fiecarei linii de cod

summary exercitiu:

În loc să păstrez parola mea ‘1234’ direct în cod, am transformat-o într-un hash cu ajutorul bcrypt. Am adăugat și un salt aleator, ca hash-ul să fie unic și să nu poată fi ghicit, chiar dacă altcineva ar avea aceeași parolă. Când eu sau altcineva introduce PIN-ul, programul ia ce am tastat, aplică același algoritm și verifică hash-ul. Dacă se potrivește cu hash-ul salvat → acces permis; dacă nu → mesaj de parolă greșită. Practic, parola mea nu e niciodată stocată clar, doar hash-ul, iar datorită salt-ului, chiar dacă altcineva are același PIN, hash-ul lui ar fi diferit și nimeni nu-l poate afla ușor

In [7]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [8]:
import bcrypt 

bcrypt conține funcții pentru a genera salt și a crea/verifica hash-uri sigure pentru parole.

Setarea parolei (de obicei doar o faci o dată și salvezi hash-ul într-o bază de date)

In [9]:
parola_corecta = "1234".encode("utf-8")  # parola originala

input() sau string-urile în Python sunt, de obicei, tip str (text). bcrypt = date brute = lucrează cu bytes, nu cu str.
#.encode("utf-8") transformă șirul "1234" într-un obiect de tip bytes, de exemplu b'1234'. De aceea facem .encode(...) înainte de a hasha.

.encode("utf-8") transformă un string într-o versiune de bytes

In [10]:
salt = bcrypt.gensalt()  # se generează un salt aleator

salt este o valoare aleatorie folosită împreună cu parola la crearea hash-ului. Scopul: două parole identice să aibă hash-uri diferite (împiedică rainbow tables).

bcrypt.gensalt() generează salt + încorporează și un cost (numit și rounds) care definește cât de „lent” e algoritmul — un cost mai mare = mai sigur dar mai lent.

Explicatie detaliata pentru mine:

În hashing cu bcrypt, două concepte cheie sunt salt și cost (rounds). Dacă doi utilizatori aleg aceeași parolă (de exemplu „1234”), fără salt hash-ul ar fi identic pentru amândoi, iar un atacator care fură baza de date ar observa imediat acest lucru și ar putea folosi rainbow tables (tabele pre-generate parolă→hash) pentru a afla parola instant. Saltul rezolvă problema: este o valoare aleatorie adăugată la parolă înainte de hashing, astfel încât și dacă doi oameni aleg aceeași parolă, hash-urile lor vor fi diferite. În bcrypt, saltul este generat automat și inclus în hash, deci nu trebuie salvat separat. Pe lângă salt, bcrypt folosește și un cost sau număr de rounds, care indică de câte ori este rulat algoritmul intern. De exemplu, un cost de 12 înseamnă aproximativ 4096 iterații, iar un cost mai mare înseamnă că hash-ul se calculează mai lent. Pentru utilizator, întârzierea e insesizabilă (milisecunde), dar pentru un atacator care încearcă milioane de parole devine foarte greu. Funcția bcrypt.gensalt() generează automat un salt unic și setează costul (implicit 12), iar rezultatul final al hashing-ului (bcrypt.hashpw) conține versiunea algoritmului, costul, saltul și hash-ul propriu-zis. Astfel, chiar dacă folosești aceeași parolă de mai multe ori, fiecare hash va arăta diferit și va fi mult mai greu de spart.

In [11]:
hashat = bcrypt.hashpw(parola_corecta, salt)

print("Hash salvat în baza de date:", hashat)

Hash salvat în baza de date: b'$2b$12$1tm6o8iy1kt4R5g7ZRAoEuRnkPRv9lXwplCtZszijO8GHVgJUXDdm'


explicatie cod de mai sus: bcrypt.hashpw() combină parola (în bytes) cu salt-ul și rulează algoritmul bcrypt pentru a produce hash-ul. Rezultatul este tot bytes.

Hash-ul generat de bcrypt include, în formatul său, informația despre versiune (ex. $2b$), cost și salt — deci hash-ul salvat conține tot ce e nevoie pentru a verifica parola mai târziu (nu trebuie să mai salvezi salt separat).

Verificarea parolei la autentificare:

In [12]:
while True:
    parola = input("Introdu PIN: ").encode("utf-8")

    if bcrypt.checkpw(parola, hashat):   # verificăm parola introdusă cu hash-ul salvat
        print("Acces permis!")
        break
    else:
        print("Parolă greșită. Încearcă din nou!")



Parolă greșită. Încearcă din nou!
Parolă greșită. Încearcă din nou!
Parolă greșită. Încearcă din nou!
Acces permis!


Explicatie cod de mai sus:
input(...) citește de la tastatură -> .encode("utf-8") transformă în bytes.

bcrypt.checkpw(candidate, stored_hash):

ia parola introdusă, extrage salt și cost din stored_hash, recalculază hash-ul și compară rezultatul cu stored_hash.

face comparația în mod sigur (pentru a evita atacuri prin diferențe de timp).

întoarce True dacă e aceeași parolă (după hashing+salt), altfel False.